In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

impoting the data -> preproccession the data

In [65]:
data = pd.read_csv("/content/drive/MyDrive/datasets/email spam/combined_data.csv")
x = data['text']
y = data['label']

1 -> means spam

0 -> means ham

splitting the data

In [66]:
x_array = np.array(x)
y_array = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x_array, y_array, test_size=0.2, random_state=42)

converting the data in [0,0,0,1,0,0,0,1] - form

In [67]:
cv = CountVectorizer(stop_words = 'english')
x_train = cv.fit_transform(x_train)
x_test = cv.transform(x_test)         # sparse matrix

In [68]:
x_train.shape

(66758, 276601)

# implimenting multinomial naive bayes from sratch

In [69]:
class MultinomialNB:
    def __init__(self):
        # initialize all model parameters
        self.classes = None
        self.class_count = {}
        self.word_count = {}
        self.feature_log_prob = {}
        self.class_log_prior = {}
        self.V = 0  # vocabulary size for laplace smoothing

    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_count = {}
        self.word_count = {}
        self.feature_log_prob = {}
        self.class_log_prior = {}
        self.V = X.shape[1]

        for c in self.classes:
            X_c = X[y == c]
            self.class_count[c] = X_c.shape[0]
            self.word_count[c] = X_c.sum(axis=0)
            total_count = self.word_count[c].sum()
            # laplace smoothing
            self.feature_log_prob[c] = np.log((self.word_count[c] + 1) / (total_count + self.V))
            self.class_log_prior[c] = np.log(self.class_count[c] / len(y))

    def predict(self, X):
        predictions = []
        for i in range(X.shape[0]):
            row = X[i]
            log_probs = {}
            for c in self.classes:
                log_prob = self.class_log_prior[c]
                log_prob += row @ self.feature_log_prob[c].T
                log_probs[c] = log_prob
            predicted_class = max(log_probs, key=log_probs.get)
            predictions.append(predicted_class)
        return np.array(predictions)

    def score(self, X, y):
        y_pred = self.predict(X)
        return np.mean(y_pred == y)

    def top_words(self, n = 5):
        # top words are with maximum p(x/y) for each class
        top_words_indices = {}
        # first geting top words
        for c in self.classes:
            top_words_indices[c] = np.argsort(self.feature_log_prob[c].A1)[-n:][::-1]

        # now getting top words for a particular class
        feature_names = cv.get_feature_names_out()
        top_words = {}
        for c in self.classes:
            top_words[c] = feature_names[top_words_indices[c]]
        return top_words


training the model

In [70]:
model = MultinomialNB()
model.fit(x_train, y_train)
print('accuracy:', model.score(x_test, y_test))

accuracy: 0.9778310365488316


In [71]:
mail = "your test is scheduled for tommorow"
vectorized = cv.transform([mail])
pred = model.predict(vectorized)
if pred == 1:
  print("spam")
else:
  print("ham")

ham


In [76]:
top_words = model.top_words(n = 10)

In [79]:
for c, words in top_words.items():
    print(f"Top words for class {c}: {', '.join(words)}")

Top words for class 0: escapenumber, http, enron, org, com, escapelong, ect, help, samba, list
Top words for class 1: escapenumber, escapelong, com, http, pills, escapenumbermg, price, company, save, item
